In [ ]:
%load_ext pystackql.magic

In [ ]:
org = "stackql"

In [ ]:
%%stackql
select name, stargazers_count 
from github.repos.repos 
where org = '$org' 
order by stargazers_count desc

In [ ]:
_.plot(kind='bar', title='Stargazers', x='name', y='stargazers_count')